In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib
matplotlib.rcParams['figure.figsize'] = [12, 6]

# Import libraries, load data

In [2]:
from pprint import pprint

import numpy as np
import pandas as pd
import patsy
import seaborn as sns
import statsmodels.api as sm
import tensorflow as tf
import tensorflow_probability as tfp

In [3]:
df = pd.read_csv(
    'https://raw.githubusercontent.com/pymc-devs/pymc3/master/pymc3/examples/data/radon.csv')

# Build models

## 1. Pooled regression model

In [4]:
# create the design matrix, with features and intercept
y, X = patsy.dmatrices(
    'log_radon ~ 1 + floor',
    data=df,
    return_type='dataframe')

# fit the model to the data
model = sm.OLS(y.iloc[:600], X.iloc[:600])
results = model.fit()

# print summary
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              log_radon   R-squared:                       0.079
Model:                            OLS   Adj. R-squared:                  0.077
Method:                 Least Squares   F-statistic:                     50.98
Date:                Wed, 10 Oct 2018   Prob (F-statistic):           2.72e-12
Time:                        21:05:59   Log-Likelihood:                -710.73
No. Observations:                 600   AIC:                             1425.
Df Residuals:                     598   BIC:                             1434.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.3962      0.035     39.362      0.000       1.327       1.466
floor         -0.6173      0.086     -7.140      0.000      -0.787      -0.448
==============================================================================
Omnibus:                       21.874   Durbin-Watson:                   1.745
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               37.581
Skew:                          -0.256   Prob(JB):                     6.91e-09
Kurtosis:                       4.114   Cond. No.                         2.76
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [5]:
# Compute inferences 
df['predict_pooled'] = results.predict(X)

mse_test = np.mean(np.square(df['predict_pooled'][600:] - df['log_radon'][600:]))
print("MSE (test set):", mse_test)

MSE (test set): 0.620216022579005


In [16]:
import tensorflow_probability as tfp

### 2. Pooled model (Bayesian)

In [9]:
from tensorflow.python.framework.ops import set_shapes_for_outputs
import edward as ed

ImportError: cannot import name 'set_shapes_for_outputs'

In [7]:
y_post = ed.copy(y, {w: qw, b: qb})

NameError: name 'ed' is not defined

In [6]:
print("Mean squared error on test data:")
print(ed.evaluate('mean_squared_error', data={X: X_test, y_post: y_test}))

print("Mean absolute error on test data:")
print(ed.evaluate('mean_absolute_error', data={X: X_test, y_post: y_test}))

Mean squared error on test data:


NameError: name 'ed' is not defined

### Model with more variables

In [ ]:
y, X = patsy.dmatrices(
    'log_radon ~ 1 + floor + Uppm',
    data=df,
    return_type='dataframe')

In [ ]:
model = sm.OLS(y.iloc[:600], X.iloc[:600])
results = model.fit()
results.summary()

In [ ]:
df['predict_pooled'] = results.predict(X)

In [ ]:
sns.jointplot('log_radon', 'predict_pooled', data=df)

In [ ]:
# make column for intercept / bias term
df['intercept'] = 1
# select feature columns for X
X_data = df[['intercept', 'adjwt', 'Uppm',
             'stfips', 'activity', 'pcterr']].values
# select output column for y
y_data = df[['log_radon']].values

In [ ]:
N, k = X_data.shape
N, k

In [ ]:
X_train, X_test = X_data[:600], X_data[600:]
y_train, y_test = y_data[:600], y_data[600:]

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
with tf.name_scope('input'):
    X = tf.placeholder(tf.float32, [None, k])
    y = tf.placeholder(tf.float32, [None, 1])
with tf.name_scope('training'):
    with tf.name_scope('model'):
        W = tf.Variable(tf.zeros([k, 1]))
        y_hat = tf.matmul(X, W)
    with tf.name_scope('loss'):
        loss = tf.reduce_mean(tf.square(y_hat - y))
        optimizer = tf.train.AdamOptimizer(learning_rate=0.0001)
        train_step = optimizer.minimize(loss)

In [ ]:
initializer = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(initializer)
    for epoch in range(0, 20000 + 1):
        sess.run(train_step, feed_dict={X: X_train, y: y_train})
        if epoch % 1000 == 0:
            y_pred = sess.run(y_hat, feed_dict={X: X_test})
            r_squared_test = 1 - \
                tf.reduce_mean(tf.square(y_pred - y_test)) / \
                tf.reduce_mean(tf.square(y_test))
            y_pred = sess.run(y_hat, feed_dict={X: X_train})
            r_squared_train = 1 - \
                tf.reduce_mean(tf.square(y_pred - y_train)) / \
                tf.reduce_mean(tf.square(y_train))
            print("Epoch: %06d | Train R^2: %06f | Test R^2: %06f" % (
                epoch,
                sess.run(r_squared_train),
                sess.run(r_squared_test)))

### 

# Evaluate results



# Appendix

### Inspect data

In [ ]:
# df['state'].value_counts()  # all MN
# df['state2'].value_counts()  # all MN
# len(df)  # 919
# df['stfips'].value_counts()  # 919
# df['zip'].value_counts()  # many zip codes (371), some clumping
# df['region'].value_counts()  # 5 regions, pretty even
# df['typebldg'].value_counts()  # mainly 1, couple others
# df['floor'].value_counts()  # usually 0, sometimes 1
# df['room'].value_counts()  # number of rooms. usually 4.

In [ ]:
df.sample(5)

In [ ]:
df['log_radon']\
    .plot(kind='hist', grid=True, legend=True, title='Distribution of radon levels')

#### Does radon level differ by `floor`?

Answer: Yes. Basements (floor 0) have more than floor 1. 

In [ ]:
ax = df\
    .groupby('floor')['log_radon']\
    .plot(
        kind='hist', grid=True, legend=True,
        title='Distribution of radon levels by floor value'
    )

In [ ]:
for c in df.columns: print(c)

In [ ]:
# verify how 'log_radon' is defined... as log(activity + 0.1)
# (df['activity'] + 0.1).apply(np.log) - df['log_radon']

In [ ]:
sns.jointplot('Uppm', 'log_radon', data=df)

In [ ]:
# verify that 'Uppm' is at county level

# df.groupby(['county', 'fips'])['Uppm'].agg(['mean', 'std', 'size']).sort_values('size', ascending=False)
# df.groupby(['county', 'fips', 'Uppm']).size().sort_values(ascending=False)